# VAE
目的：
- 输入：MNIST 手写数字图像（28x28）
- 编码器 → 学到隐变量 z（概率分布）
- 解码器 → 从 z 重建图像
最后可生成新图像、学习潜在空间结构   

整体流程：   
1. 先定义模型结构 VAE
2. 然后定义损失函数 loss_function
3. 接着写训练代码 for epoch in ...（通常写在 main 里或者 notebook 后半部分）
4. 在 model(x) 这一步自动触发 forward()（PyTorch 会自动调用）
5. 用返回的值算 loss → 用 loss 来训练模型

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os

这里的`input_dim=784`就是根据MNIST手写数据图像的尺寸来确定的
## 1.  定义VAE模型
### 1.1 编码器结构
第一步：图像（784维） → 隐藏层（400维）
然后：输出两个东西 → `μ` 和 `log(σ²)`（代表分布）
### 1.2 解码器结构
从 latent 向量 z → 还原图像（输出784维）
sigmoid 保证输出像素值在 `[0,1]`
z（20维）
 ↓ fc2（20→400）+ ReLU
隐藏层 h（400维）
 ↓ fc3（400→784）+ Sigmoid
还原图像（784维）

### 1.3 重参数操作
你想象一下我们写个网络这样：
```python
mu, logvar = encoder(x)
z = torch.normal(mu, sigma)  # ← 你以为可以这么搞
x_hat = decoder(z)
```
这样写在数学上没法对 mu 和 sigma 求导数，也就是你没法告诉 encoder “你输出的 mu 应该改成啥”。这样网络就没法训练！   

我们用重参数技巧（Reparameterization Trick）来绕开这个问题, `z=μ+σ⋅ϵ,ϵ∼N(0,1)`。   
从 𝑁(μ,σ^2) 采样 𝑧不合适，为啥不直接从 N(μ,σ^2) 里采样 z？
因为直接采样会导致梯度无法传播，`reparameterization trick` 通过引入一个随机变量ϵ来解决这个问题。
- ϵ是从标准正态分布N(0,1)中采样的随机变量, `ϵ∼N(0,1)`，然后通过公式 z=μ+σ⋅ϵ 来生成潜在变量z。
- 这里的σ是通过对数方差logvar计算得到的，具体为 `σ=exp(0.5*logvar)`。
- 这里的μ是编码器输出的均值
这样就可以将随机性引入到模型中，同时又能保持梯度的可传播性。

In [ ]:
from logging import NullHandler
# 1. 定义VAE模型
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        super(VAE, self).__init__()
        # 编码器部分
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # 解码器部分
        self.fc2 = nn.Linear(latent_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)

    # encoder: 输入数据x，输出均值mu和对数方差logvar
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc_mu(h), self.fc_logvar(h)

    # reparameterize 重参数技巧: 使用均值和对数方差生成潜在变量z
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    # decode: 使用潜在变量z生成重构数据
    def decode(self, z):
        h = F.relu(self.fc2(z))
        return torch.sigmoid(self.fc3(h))

    # forward: 整体前向传播过程，返回重构数据、均值和对数方差
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

#### 重参数小例子
运行这个例子会看到如下结果：   
mu: [1.0, -0.5]   
std: [1.0, 0.1353]        ← 注意 std 是从 logvar 还原出来的   
epsilon: [0.62, -0.18]    ← 随机生成的 ε   
z: [1.62, -0.5243]        ← z = mu + std * eps   

In [ ]:
import torch

# 模拟编码器输出
mu = torch.tensor([1.0, -0.5])           # 均值
logvar = torch.tensor([0.0, -2.0])       # 对数方差

# 重参数技巧实现 z = mu + sigma * epsilon
std = torch.exp(0.5 * logvar)            # 方差开根号 = 标准差
eps = torch.randn_like(std)             # 从 N(0,1) 采样 ε
z = mu + std * eps                      # 最终采样出的潜变量 z

print("mu:", mu.tolist())
print("std:", std.tolist())
print("epsilon:", eps.tolist())
print("z:", z.tolist())

# 2. 定义VAE的损失函数（重建 + KL散度）

In [3]:
# 2. 定义VAE的损失函数（重建 + KL散度）
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [4]:
# 3. 准备训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 4. 加载MNIST数据
transform = transforms.ToTensor()
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=128, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.61MB/s]


In [ ]:
# 5. 训练模型
model.train()
for epoch in range(1, 6):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.view(-1, 784).to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {train_loss / len(train_loader.dataset):.4f}")

Epoch 1, Loss: 163.2387
Epoch 2, Loss: 120.6043
Epoch 3, Loss: 114.2044
Epoch 4, Loss: 111.3464
Epoch 5, Loss: 109.6627


In [ ]:
# 6. 随机生成新图像
model.eval()
with torch.no_grad():
    z = torch.randn(64, 20).to(device)
    sample = model.decode(z).cpu().view(64, 1, 28, 28)
    os.makedirs("vae_output", exist_ok=True)
    save_image(sample, 'vae_output/sample.png')
    print("新图像已保存为 vae_output/sample.png")


新图像已保存为 vae_output/sample.png
